<a href="https://colab.research.google.com/github/MahdiTheGreat/Game-playing-systems/blob/main/TicTacToe_Heuristics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import time
import random
from IPython.display import clear_output
from google.colab import output



class TicTacToe:
    def __init__(self):
        # Initialize an empty 3x3 board
        self.board = np.array([[' ' for _ in range(3)] for _ in range(3)])
        self.current_player = 'O'  # AI player is O and starts first
        self.ai_player = 'O'       # AI player is O
        self.human_player = 'X'    # Human player is X

        # Winning combinations (rows, columns, diagonals)
        self.winning_combos = [
            [(0,0), (0,1), (0,2)], [(1,0), (1,1), (1,2)], [(2,0), (2,1), (2,2)],  # rows
            [(0,0), (1,0), (2,0)], [(0,1), (1,1), (2,1)], [(0,2), (1,2), (2,2)],  # columns
            [(0,0), (1,1), (2,2)], [(0,2), (1,1), (2,0)]  # diagonals
        ]

        # Strategic positions with weights
        self.position_weights = {
            (1,1): 4,  # Center (highest value)
            (0,0): 3, (0,2): 3, (2,0): 3, (2,2): 3,  # Corners (second highest)
            (0,1): 2, (1,0): 2, (1,2): 2, (2,1): 2   # Edges (lowest value)
        }

    def display_board(self):
        """Display the current state of the board"""
        print('  0 1 2')
        for i in range(3):
            print(f"{i} {self.board[i][0]}|{self.board[i][1]}|{self.board[i][2]}")
            if i < 2:
                print("  -+-+-")
        print("\n")

    def is_valid_move(self, row, col):
        """Check if a move is valid"""
        if row < 0 or row > 2 or col < 0 or col > 2:
            return False
        return self.board[row][col] == ' '

    def make_move(self, row, col, player):
        """Make a move on the board"""
        if self.is_valid_move(row, col):
            self.board[row][col] = player
            return True
        return False

    def check_winner(self):
        """Check if there is a winner"""
        # Check rows
        for i in range(3):
            if self.board[i][0] != ' ' and self.board[i][0] == self.board[i][1] == self.board[i][2]:
                return self.board[i][0]

        # Check columns
        for i in range(3):
            if self.board[0][i] != ' ' and self.board[0][i] == self.board[1][i] == self.board[2][i]:
                return self.board[0][i]

        # Check diagonals
        if self.board[0][0] != ' ' and self.board[0][0] == self.board[1][1] == self.board[2][2]:
            return self.board[0][0]
        if self.board[0][2] != ' ' and self.board[0][2] == self.board[1][1] == self.board[2][0]:
            return self.board[0][2]

        # Check if the board is full (draw)
        if ' ' not in self.board.flatten():
            return 'Draw'

        return None

    def is_fork_possible(self, player):
        """Check if a fork is possible and return the position"""
        # A fork is a move that creates two winning opportunities
        empty_cells = [(i, j) for i in range(3) for j in range(3) if self.board[i][j] == ' ']

        for i, j in empty_cells:
            # Try this move
            self.board[i][j] = player

            # Count possible winning lines
            winning_opportunities = 0
            for combo in self.winning_combos:
                combo_values = [self.board[r][c] for r, c in combo]
                # If this combo has exactly 2 of the player's symbols and 1 empty, it's a winning opportunity
                if combo_values.count(player) == 2 and combo_values.count(' ') == 1:
                    winning_opportunities += 1

            # Undo the move
            self.board[i][j] = ' '

            # If this creates 2 or more winning opportunities, it's a fork
            if winning_opportunities >= 2:
                return (i, j)

        return None

    def get_opposite_corner(self, row, col):
        """Get the opposite corner position"""
        if (row, col) == (0, 0):
            return (2, 2)
        elif (row, col) == (0, 2):
            return (2, 0)
        elif (row, col) == (2, 0):
            return (0, 2)
        elif (row, col) == (2, 2):
            return (0, 0)
        return None

    def create_fork_block(self):
        """Create a forcing move that blocks potential forks"""
        # Find all empty cells
        empty_cells = [(i, j) for i in range(3) for j in range(3) if self.board[i][j] == ' ']

        for i, j in empty_cells:
            # Try this move
            self.board[i][j] = self.ai_player

            # If this move forces the human to block, and doesn't allow them to create a fork, it's good
            block_required = self.find_winning_move(self.ai_player)

            # Simulate human blocking
            if block_required:
                self.board[block_required[0]][block_required[1]] = self.human_player

                # Check if human can fork after blocking
                fork_possible = self.is_fork_possible(self.human_player)

                # Undo the block
                self.board[block_required[0]][block_required[1]] = ' '

                if not fork_possible:
                    # Undo the test move
                    self.board[i][j] = ' '
                    return (i, j)

            # Undo the test move
            self.board[i][j] = ' '

        return None

    def find_winning_move(self, player):
        """Find a winning move for the given player"""
        for combo in self.winning_combos:
            values = [self.board[r][c] for r, c in combo]
            positions = [pos for pos, val in zip(combo, values) if val == ' ']

            if values.count(player) == 2 and values.count(' ') == 1:
                # This is a winning opportunity - return the empty position
                return positions[0]

        return None

    def get_best_strategic_position(self):
        """Get the best available position based on strategic weights"""
        # Get all empty positions with their weights
        weighted_positions = {pos: weight for pos, weight in self.position_weights.items()
                               if self.is_valid_move(pos[0], pos[1])}

        if not weighted_positions:
            return None

        # Get positions with the highest weight
        max_weight = max(weighted_positions.values())
        best_positions = [pos for pos, weight in weighted_positions.items() if weight == max_weight]

        # Randomly choose one of the best positions
        return random.choice(best_positions)

    def ai_move(self):
        """Make a move for the AI using improved heuristics"""
        # 1. Win if possible
        winning_move = self.find_winning_move(self.ai_player)
        if winning_move:
            return winning_move

        # 2. Block opponent's winning move
        blocking_move = self.find_winning_move(self.human_player)
        if blocking_move:
            return blocking_move

        # 3. Create a fork (two winning ways)
        fork_move = self.is_fork_possible(self.ai_player)
        if fork_move:
            return fork_move

        # 4. Block opponent's fork
        opponent_fork = self.is_fork_possible(self.human_player)
        if opponent_fork:
            return opponent_fork

        # 5. Try to create a forcing move that prevents forks
        fork_block = self.create_fork_block()
        if fork_block:
            return fork_block

        # 6. Take center if it's the first move or if it's available
        if self.is_valid_move(1, 1):
            return (1, 1)

        # 7. Take opposite corner of opponent's piece
        for corner in [(0, 0), (0, 2), (2, 0), (2, 2)]:
            if self.board[corner[0]][corner[1]] == self.human_player:
                opposite = self.get_opposite_corner(corner[0], corner[1])
                if opposite and self.is_valid_move(opposite[0], opposite[1]):
                    return opposite

        # 8. Take any corner or edge based on strategic weights
        return self.get_best_strategic_position()

# Function for each turn (solves Colab input issues)
def play_turn(game):
    # Clear previous output
    output.clear()

    # AI's turn
    if game.current_player == 'O':
        print("AI is thinking...")
        time.sleep(1)

        row, col = game.ai_move()
        game.make_move(row, col, 'O')

        # Display board after AI move
        game.display_board()
        print(f"AI chose: {row} {col}")

        game.current_player = 'X'  # Switch to human's turn

    # Check if game is over after AI's move
    winner = game.check_winner()
    if winner:
        if winner == 'X':
            print("Congratulations! You won!")
        elif winner == 'O':
            print("AI wins! Better luck next time.")
        else:
            print("It's a draw!")
        return False

    # Human's turn
    if game.current_player == 'X':
        print("Your turn (X)")
        try:
            row = int(input("Enter row (0-2): "))
            col = int(input("Enter column (0-2): "))

            if game.make_move(row, col, 'X'):
                game.current_player = 'O'  # Switch to AI's turn
            else:
                print("Invalid move! Try again.")
                time.sleep(2)
                return True
        except (ValueError, IndexError):
            print("Invalid input! Please enter row and column as numbers from 0-2.")
            time.sleep(2)
            return True

    # Check if game is over after human's move
    winner = game.check_winner()
    if winner:
        output.clear()
        game.display_board()
        if winner == 'X':
            print("Congratulations! You won!")
        elif winner == 'O':
            print("AI wins! Better luck next time.")
        else:
            print("It's a draw!")
        return False

    return True

# Main game function1
def main():
    print("Welcome to Tic Tac Toe!")
    print("You are X, the AI is O.")
    print("AI goes first!")
    print("To make a move, enter row (0-2) and column (0-2).")
    time.sleep(2)

    game = TicTacToe()

    # Start first turn
    keep_playing = True

    while keep_playing:
        keep_playing = play_turn(game)

# Run the game
main()

AI is thinking...
  0 1 2
0 X|O|O
  -+-+-
1 O|O|X
  -+-+-
2 X|X|O


AI chose: 0 2
It's a draw!
